In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [4]:
data=pd.read_csv('/content/drive/MyDrive/Dataset/ipl2021/matches.csv')

In [5]:
df=data[(data['team1']=='Mumbai Indians') | (data['team2']=='Mumbai Indians')]
MI=df.iloc[:]
MI.head()

,id,Season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
1,2,IPL-2017,Pune,06-04-2017,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
6,7,IPL-2017,Mumbai,09-04-2017,Kolkata Knight Riders,Mumbai Indians,Mumbai Indians,field,normal,0,Mumbai Indians,0,4,N Rana,Wankhede Stadium,Nitin Menon,CK Nandan,NaN
9,10,IPL-2017,Mumbai,12-04-2017,Sunrisers Hyderabad,Mumbai Indians,Mumbai Indians,field,normal,0,Mumbai Indians,0,4,JJ Bumrah,Wankhede Stadium,Nitin Menon,CK Nandan,NaN
11,12,IPL-2017,Bangalore,14-04-2017,Royal Challengers Bangalore,Mumbai Indians,Mumbai Indians,field,normal,0,Mumbai Indians,0,4,KA Pollard,M Chinnaswamy Stadium,KN Ananthapadmanabhan,AK Chaudhary,NaN
15,16,IPL-2017,Mumbai,16-04-2017,Gujarat Lions,Mumbai Indians,Mumbai Indians,field,normal,0,Mumbai Indians,0,6,N Rana,Wankhede Stadium,A Nand Kishore,S Ravi,NaN


In [7]:
TotalTeams = ['Mumbai Indians', ' Kings XI Punjab', 'Chennai Super Kings', 'Royal Challengers Bangalore', 
            'Kolkata Knight Riders', 'Delhi Daredevils', 'Rajasthan Royals', 'Sunrisers Hyderabad']
T1=data[data['team1'].isin(TotalTeams)]
T2=data[data['team2'].isin(TotalTeams)] 
teams=pd.concat((T1,T2))
teams.drop_duplicates()
teams.count()

id                 1125
Season             1125
city               1112
date               1125
team1              1125
team2              1125
toss_winner        1125
toss_decision      1125
result             1125
dl_applied         1125
winner             1118
win_by_runs        1125
win_by_wickets     1125
player_of_match    1118
venue              1125
umpire1            1122
umpire2            1122
umpire3             195
dtype: int64

In [8]:
team_2010=teams.drop(['id','Season','city','date','toss_winner','toss_decision','dl_applied','win_by_runs','win_by_wickets','result','player_of_match','venue','umpire1','umpire2','umpire3'],axis=1)
team_2010.head()

,team1,team2,winner
0,Sunrisers Hyderabad,Royal Challengers Bangalore,Sunrisers Hyderabad
1,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant
4,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore
6,Kolkata Knight Riders,Mumbai Indians,Mumbai Indians
7,Royal Challengers Bangalore,Kings XI Punjab,Kings XI Punjab


In [9]:
team_2010=team_2010.reset_index(drop=True)
team_2010.loc[team_2010.winner==team_2010.team1,'winning_team']=1
team_2010.loc[team_2010.winner==team_2010.team2,'winning_team']=2
team_2010=team_2010.drop(['winning_team'],axis=1)
team_2010.head()


,team1,team2,winner
0,Sunrisers Hyderabad,Royal Challengers Bangalore,Sunrisers Hyderabad
1,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant
2,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore
3,Kolkata Knight Riders,Mumbai Indians,Mumbai Indians
4,Royal Challengers Bangalore,Kings XI Punjab,Kings XI Punjab


In [10]:
team_2010=team_2010.dropna()
team_2010.count()

team1     1118
team2     1118
winner    1118
dtype: int64

In [11]:
final=pd.get_dummies(team_2010,prefix=['team1','team2'],columns=['team1','team2'])
X=final.drop(['winner'],axis=1)
y=final.winner
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)



In [13]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, max_depth=20,random_state=0) 
rf.fit(X_train,y_train)
score = rf.score(X_train, y_train)
score2 = rf.score(X_test, y_test)
print("Training set accuracy: ", '%.3f'%(score))
print("Test set accuracy: ", '%.3f'%(score2))

Training set accuracy:  0.664
Test set accuracy:  0.586


In [14]:
from sklearn.svm import SVC  
clf = SVC(kernel='linear') 
clf.fit(X_train,y_train)
score = clf.score(X_train, y_train)
score2 = clf.score(X_test, y_test)
print("Training set accuracy: ", '%.3f'%(score))
print("Test set accuracy: ", '%.3f'%(score2))

Training set accuracy:  0.655
Test set accuracy:  0.589


In [15]:
ranking = pd.read_csv('/content/drive/MyDrive/Dataset/ipl2021/ranking.csv') 
fixtures = pd.read_csv('/content/drive/MyDrive/Dataset/ipl2021/matches.csv')
pred_set = []

In [16]:
fixtures.insert(1, 'first_position', fixtures['team1'].map(ranking.set_index('team')['ranking']))
fixtures.insert(2, 'second_position', fixtures['team2'].map(ranking.set_index('team')['ranking']))

fixtures = fixtures.iloc[:5, :]
fixtures.tail()

,id,first_position,second_position,Season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,5.0,6.0,IPL-2017,Hyderabad,05-04-2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,1.0,NaN,IPL-2017,Pune,06-04-2017,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,NaN,3.0,IPL-2017,Rajkot,07-04-2017,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,NaN,8.0,IPL-2017,Indore,08-04-2017,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,6.0,7.0,IPL-2017,Bangalore,08-04-2017,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [17]:
for index, row in fixtures.iterrows():
    if row['first_position'] < row['second_position']:
        pred_set.append({'team1': row['team1'], 'team2': row['team2'], 'winning_team': None})
    else:
        pred_set.append({'team1': row['team1'], 'team2': row['team2'], 'winning_team': None})
        
pred_set = pd.DataFrame(pred_set)
backup_pred_set = pred_set
pred_set.head()

,team1,team2,winning_team
0,Sunrisers Hyderabad,Royal Challengers Bangalore,None
1,Mumbai Indians,Rising Pune Supergiant,None
2,Gujarat Lions,Kolkata Knight Riders,None
3,Rising Pune Supergiant,Kings XI Punjab,None
4,Royal Challengers Bangalore,Delhi Daredevils,None


In [18]:

pred_set = pd.get_dummies(pred_set, prefix=['Team_1', 'Team_2'], columns=['team1', 'team2'])

missing_cols = set(final.columns) - set(pred_set.columns)
for c in missing_cols:
    pred_set[c] = 0
pred_set = pred_set[final.columns]


pred_set = pred_set.drop(['winner'], axis=1)
pred_set.head()

,team1_Chennai Super Kings,team1_Deccan Chargers,team1_Delhi Capitals,team1_Delhi Daredevils,team1_Gujarat Lions,team1_Kings XI Punjab,team1_Kochi Tuskers Kerala,team1_Kolkata Knight Riders,team1_Mumbai Indians,team1_Pune Warriors,team1_Rajasthan Royals,team1_Rising Pune Supergiant,team1_Rising Pune Supergiants,team1_Royal Challengers Bangalore,team1_Sunrisers Hyderabad,team2_Chennai Super Kings,team2_Deccan Chargers,team2_Delhi Capitals,team2_Delhi Daredevils,team2_Gujarat Lions,team2_Kings XI Punjab,team2_Kochi Tuskers Kerala,team2_Kolkata Knight Riders,team2_Mumbai Indians,team2_Pune Warriors,team2_Rajasthan Royals,team2_Rising Pune Supergiant,team2_Rising Pune Supergiants,team2_Royal Challengers Bangalore,team2_Sunrisers Hyderabad
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
predictions = rf.predict(pred_set)
for i in range(fixtures.shape[0]):
    print(backup_pred_set.iloc[i, 1] + " and " + backup_pred_set.iloc[i, 0])
    if predictions[i] == 1:
        print("Winner: " + backup_pred_set.iloc[i, 1])
    
    else:
        print("Winner: " + backup_pred_set.iloc[i, 0])
    print("")

Royal Challengers Bangalore and Sunrisers Hyderabad
Winner: Sunrisers Hyderabad

Rising Pune Supergiant and Mumbai Indians
Winner: Mumbai Indians

Kolkata Knight Riders and Gujarat Lions
Winner: Gujarat Lions

Kings XI Punjab and Rising Pune Supergiant
Winner: Rising Pune Supergiant

Delhi Daredevils and Royal Challengers Bangalore
Winner: Royal Challengers Bangalore



In [20]:
semi = [('Royal Challengers Bangalore', 'Sunrisers Hyderabad'),
            ('Mumbai Indians', 'Chennai Super Kings')]

In [21]:
def clean_and_predict(matches, ranking, final, rf):
    positions = []
    for match in matches:
        positions.append(ranking.loc[ranking['team'] == match[0],'ranking'].iloc[0])
        positions.append(ranking.loc[ranking['team'] == match[1],'ranking'].iloc[0])

    pred_set = []

    i = 0
    j = 0

    while i < len(positions):
        dict1 = {}

        if positions[i] < positions[i + 1]:
            dict1.update({'team1': matches[j][0], 'team2': matches[j][1]})
        else:
            dict1.update({'team1': matches[j][1], 'team2': matches[j][0]})

        pred_set.append(dict1)
        i += 2
        j += 1
        
    pred_set = pd.DataFrame(pred_set)
    backup_pred_set = pred_set
    pred_set = pd.get_dummies(pred_set, prefix=['team1', 'team2'], columns=['team1', 'team2'])
    missing_cols2 = set(final.columns) - set(pred_set.columns)
    for c in missing_cols2:
        pred_set[c] = 0
    pred_set = pred_set[final.columns]

    pred_set = pred_set.drop(['winner'], axis=1)

    predictions = rf.predict(pred_set)
    for i in range(len(pred_set)):
        print(backup_pred_set.iloc[i, 1] + " and " + backup_pred_set.iloc[i, 0])
        if predictions[i] == 1:
            print("Winner: " + backup_pred_set.iloc[i, 1])
        else:
            print("Winner: " + backup_pred_set.iloc[i, 0])
        print("")

In [22]:
clean_and_predict(semi, ranking, final, rf)

Royal Challengers Bangalore and Sunrisers Hyderabad
Winner: Sunrisers Hyderabad



In [ ]:
finals = [('Sunrisers Hyderabad', 'Mumbai Indians')]
clean_and_predict(finals, ranking, final, rf)

Sunrisers Hyderabad and Mumbai Indians
Winner: Mumbai Indians

